## Training Notebook

This notebook is a tool to train the neural network for binary classification using the LFW Dataset.

In [ ]:
# Import thrid party libraries

import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Import utility libraries

from src.utils.triplet_generator import TripletGenerator

# Import the core library from Precencia

from src.core.face_encoder import FaceEncoder

In [ ]:
# Initialize utility libraries

triplet_generator = TripletGenerator()

# Initialize the classes

face_encoder = FaceEncoder()

In [ ]:
# Generate triplets pairs to ease the setting up the data for training

triplet_generator.lfw_dataset_source('\data\lfw')
(anchors, positives, negatives) = triplet_generator.create_triplets()

In [ ]:
triplet_generator.dataset_info()

In [ ]:
# Feature extraction to get face embedding vectors, in (anchor, positive) and (anchor, negative) pair

pair_embeddings = []
y = []

length = len(anchors)

for idx in range(length):
  anchor_embedding = face_encoder.get_embedding(image_path=anchors[idx])
  positive_embedding = face_encoder.get_embedding(image_path=positives[idx])
  negative_embedding = face_encoder.get_embedding(image_path=negatives[idx])

  pair_embeddings.append((anchor_embedding, positive_embedding))
  y.append(1)
  pair_embeddings.append((anchor_embedding, negative_embedding))
  y.append(0)


np.save('./data/encodings/lfw_face_embeddings.npy', pair_embeddings)

In [ ]:
# Load face embedding pairs

pair_embeddings = np.load('./data/encodings/lfw_face_embeddings.npy')

In [ ]:
X = []
y = []

length = len(anchors)

for _ in range(length):
  y.append(1)
  y.append(0)

for embedding in pair_embeddings:
  X.append(np.array(embedding[0] - embedding[1]).reshape(-1))

X_train = np.array(X)
y_train = np.array(y).reshape(-1)

In [ ]:
def classifier_scheduler(epoch, lr):
  if (epoch % 5 == 0):
    return lr * tf.math.exp(-0.1)
  
  return lr

ClassifierLearningRateScheduler = LearningRateScheduler(classifier_scheduler, verbose=1)
ClassifierCustomEarlyStopping = EarlyStopping(monitor='loss', patience=3)

classifier = Sequential([
            Input(shape=(128,)),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])

In [ ]:
classifier.compile(loss='binary_crossentropy', optimizer=Adam(0.002), metrics=['accuracy'])
classifier_training_history = classifier.fit(X_train, y_train, validation_split=0.2, callbacks=[
               ClassifierLearningRateScheduler, ClassifierCustomEarlyStopping], epochs=20, verbose=1)

In [ ]:
print(classifier_training_history.history.keys())

plt.plot(classifier_training_history.history['accuracy'])
plt.plot(classifier_training_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(classifier_training_history.history['loss'])
plt.plot(classifier_training_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(classifier_training_history.history['lr'])
plt.title('Learning rate')
plt.ylabel('learning rate')
plt.xlabel('epoch')
plt.legend(['lr'], loc='upper left')
plt.show()

classifier.summary()

In [ ]:
classifier.save("./data/models/classifier_keras_weights.h5")